In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import autograd
import torch.nn.functional as F

images = np.load("D:/work/JHUschoolStuff/machinelearning/project1/cs475_project_data/images.npy")
labels = np.load("D:/work/JHUschoolStuff/machinelearning/project1/cs475_project_data/labels.npy")
test = np.load("D:/work/JHUschoolStuff/machinelearning/project1/cs475_project_data/test_images.npy")
height = images.shape[1]
width = images.shape[2]
size = height * width
images = (images - images.mean()) / images.std()
data = images.reshape(images.shape[0],size)
test_data = test.reshape(test.shape[0], size)
test_data = (test_data - test_data.mean()) / test_data.std()
batch_size = 1
NUM_OPT_STEPS = 5000
train_seqs, train_labels = data[0:45000,:], labels[0:45000]
val_seqs, val_labels = data[45000:,:], labels[45000:]

In [2]:
class TwoLayerNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = torch.nn.Linear(height * width, 100)
        self.layer_2 = torch.nn.Linear(100, 5)
    def forward(self, x):
        x = self.layer_1(x)
        y = F.relu(x)
        z = self.layer_2(y)
        return z

In [3]:
model = TwoLayerNN()

In [4]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [5]:
def train(batch_size):
    # model.train() puts our model in train mode, which can require different
    # behavior than eval mode (for example in the case of dropout).
    model.train()
    # i is is a 1-D array with shape [batch_size]
    i = np.random.choice(train_seqs.shape[0], size=batch_size, replace=False)
    x = autograd.Variable(torch.from_numpy(train_seqs[i].astype(np.float32)))
    y = autograd.Variable(torch.from_numpy(train_labels[i].astype(np.int))).long()
    optimizer.zero_grad()
    y_hat_ = model(x)
    loss = F.cross_entropy(y_hat_, y)
    loss.backward()
    optimizer.step()
    return loss.data[0]


In [6]:
def accuracy(y, y_hat):
    return (y == y_hat).astype(np.float).mean()

In [7]:
def approx_train_accuracy(model):
    i = np.random.choice(train_seqs.shape[0], size=1000, replace=False)
    x = autograd.Variable(torch.from_numpy(train_seqs[i].astype(np.float32)))
    y = autograd.Variable(torch.from_numpy(train_labels[i].astype(np.int)))
    y_hat_ = model(x)
    pred = []
    for j in range(y_hat_.size()[0]):
        logits = y_hat_[j,:].data.numpy()
        pred.append(np.argmax(logits))
    return accuracy(pred, y.data.numpy())

In [8]:
def val_accuracy(model):
    x = autograd.Variable(torch.from_numpy(val_seqs.astype(np.float32)))
    y = autograd.Variable(torch.from_numpy(val_labels.astype(np.int)))
    y_hat_ = model(x)
    pred = []
    for j in range(y_hat_.size()[0]):
        logits = y_hat_[j,:].data.numpy()
        pred.append(np.argmax(logits))
    return accuracy(pred, y.data.numpy())

In [9]:
def plot_accuracies(tr, v):
    ind = list(range(len(tr)))
    plt.plot(ind,tr,'-ro')
    plt.title('Training accuracy as a function of iteration')
    plt.xlabel('iteration')
    plt.ylabel('training accuracy')
    plt.show()
    plt.plot(ind,v,'-go')
    plt.title('Validation accuracy as a function of iteration')
    plt.xlabel('iteration')
    plt.ylabel('validation accuracy')
    plt.show()

In [10]:
batch_size = 1
train_accs, val_accs = [], []
for i in range(NUM_OPT_STEPS):
    train(batch_size)
    if i % 100 == 0:
        train_accs.append(approx_train_accuracy(model))
        val_accs.append(val_accuracy(model))
        print("%6d %5.2f %5.2f" % (i, train_accs[-1], val_accs[-1]))
plot_accuracies(train_accs, val_accs)

KeyboardInterrupt: 

In [ ]:
for m in model.children():
    m.reset_parameters()
#optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
batch_size = 10
train_accs, val_accs = [], []
for i in range(NUM_OPT_STEPS):
    train(batch_size)
    if i % 100 == 0:
        train_accs.append(approx_train_accuracy(model))
        val_accs.append(val_accuracy(model))
        print("%6d %5.2f %5.2f" % (i, train_accs[-1], val_accs[-1]))
plot_accuracies(train_accs, val_accs)